In [3]:
"""Update Salesforce data from Google Sheets."""
import json
import pandas as pd
from salesforce_bulk import SalesforceBulk, CsvDictsAdapter
from salesforce_bulk.util import IteratorBytesIO
from utils.setup import log, vault_credentials
from utils import gs
from datetime import datetime
import httplib2
import os
from apiclient import discovery
from oauth2client import client, tools
from oauth2client.file import Storage

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def get_credentials():
    """Gets valid user credentials from storage.
    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.
    Returns:
        Credentials, the obtained credential.
    """
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    CLIENT_SECRET_FILE = 'client_secret.json'

    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                   'sheets.googleapis.com-python-quickstart.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    return credentials

In [4]:
def buildGS():
    credentials = get_credentials()
    http = credentials.authorize(httplib2.Http())
    discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                    'version=v4')
    service = discovery.build('sheets', 'v4', http=http,
                              discoveryServiceUrl=discoveryUrl)
    return service

In [119]:
def get_folder_contents(folder_id):
    credentials = get_credentials()
    http = credentials.authorize(httplib2.Http())
    service = discovery.build('drive', 'v3', http=http)

    results = service.files().list(
        q=(f"'{folder_id}' in parents and trashed = false"),
#         corpora="domain",
        fields="files(id, name)",
        supportsTeamDrives=True,
        includeTeamDriveItems=True).execute()
    items = results.get('files', [])
    if not items:
        print('No files found.')
    return items

In [6]:
def get_archived_shiftplan_sheet(date):
    
    #this is the folder where the archived plans are
    for folder in get_folder_contents('gsheet_id'): 
        
        #grabs the folder of the corresponding year of the date
        if folder['name'] == date.strftime('%Y'):

            #gets the shiftplans sheets from that year
            shiftplan_folders = get_folder_contents(folder['id'])

            for files in shiftplan_folders:

                #gets metadata from all files in this folder
                sheets_data = buildGS().spreadsheets().get(spreadsheetId=files['id']).execute()

                #checks inside each sheet for the tab that contains the specific date
                for tab in sheets_data['sheets']:

                    if tab['properties']['title'] == date.strftime('%d.%m.%Y'):

                        #saves the id from the sheet containing the searched date and returns it
                        sheet_id = sheets_data['spreadsheetId']
                        return sheet_id
    return 'sheet not found'
    


In [84]:
def get_gs_to_df(sheet_id, tab_name):
    gs = buildGS().spreadsheets().values().get(spreadsheetId=sheet_id, range=tab_name.strftime('%d.%m.%Y')).execute()
    gs = pd.DataFrame(data=gs['values']) #load google sheet into df
    gs = gs.iloc[:,:68] #exclude empty columns
    gs.set_index(gs.iloc[:,1], inplace=True) #set specialist names as index
    gs = gs.iloc[:gs.index.get_loc('T'),:]
    delete_list = [1,2] #include by default row 1 and 2 to delete which are superfluous
    gs.fillna(value="", inplace=True)
    for i,index in enumerate(gs.index):
        if len(index) < 1 | len(index)==None:
            delete_list.append(i) #delete empty rows
        elif not (
            " " in index
            and "exclude" not in index
            and "this" not in index
            and "and" not in index
            and "that" not in index
        ):
            delete_list.append(i) #delete all rows that are not names of specialists

    delete_list.remove(0) #except first row which has the hours
    gs.drop(gs.index[delete_list], inplace=True)
    gs.rename({i: v  for i,v in enumerate(gs.iloc[0,:])}, axis=1, inplace=True) #rename columns with matching times
    gs = gs.T #transpose df
    gs.drop(gs.index[0], inplace=True) #drop language row, to be possibly included in future iterations
    gs.iloc[:,0] = datetime.strftime(datetime.strptime(gs.iloc[:,0][0], '%d/%m/%Y'),'%m/%d/%Y')
    gs.rename({f'{gs.index[0]}': gs.index[0].replace('/','.')}, inplace=True)
    gs.reset_index(inplace=True)
    gs.rename(columns={'index': gs.iloc[0,0], f'{gs.columns[1]}': str(gs.iloc[0,1])}, inplace=True)
    gs.iloc[1:,0] = [datetime.strftime(datetime.strptime(i,'%H:%M'),'%H:%M') for i in gs.iloc[1:,0]]
    return gs

In [29]:
def df_to_pre_SF(df):
    # exclude columns that are not specialist names
    for col in df.columns[2:]:
        check_val = df.loc[0, col]
        if not (
            " " in check_val
            and "some value" not in check_val
            and "another value" not in check_val
            and "yav" not in check_val
            and "value" not in check_val
        ):
            df.drop(columns=[col], inplace=True)

    # use first column as column names
    df.columns = df.iloc[0]
    # delete first column
    df = df.reindex(df.index.drop(0))

    # rename first column
    first_two_cols = ["time", "date"]
    df.columns.values[[0, 1]] = first_two_cols

    df = df.set_index(first_two_cols)
    # turn dataframe into series where previous columns are index, drop nas
    df = df.stack()

    # rename indexes
    df.index.names = first_two_cols + ["name"]

    # turn indexes back to columns
    df = df.reset_index()

    # create timestamp column and convert to utc
    df.loc[:, "timestamp"] = pd.to_datetime(
        df["date"] + df["time"], format="%m/%d/%Y%H:%M"
    )
    df["timestamp"] = (
        df["timestamp"].dt.tz_localize("Europe/Berlin").dt.tz_convert("UTC")
    )
    df["timestamp"] = df["timestamp"].dt.strftime("%Y-%m-%dT%H:%M:%S.000Z")

    # rename columns to desired final name
    df = df.rename(
        index=str,
        columns={
            0: "column1__c",
            "timestamp": "column2__c",
            "name": "column3__c",
        },
    )

    #drop rows with no activity (null or empty)
    df = df[df['column1__c'].apply(len) > 0]
    return df

In [9]:
def sf_bulk_job(bulk, object, query=False, insert=False, delete=False):
    """Wrapper for SF bulk operations."""
    # Test that one and only one type of action was specified
    if sum(map(bool, [query, insert, delete])) != 1:
        raise ValueError(
            "Please specify exactly one option - either query, insert or delete."
        )

    # initiate correct job
    if query:
        job = bulk.create_query_job(object, contentType="JSON")
        batch = bulk.query(job, query)
    else:
        if insert:
            job = bulk.create_insert_job(object, contentType="CSV")
            batch_list = insert
        elif delete:
            job = bulk.create_delete_job(object, contentType="CSV")
            batch_list = delete

        csv_iter = CsvDictsAdapter(iter(batch_list))
        batch = bulk.post_batch(job, csv_iter)

    bulk.wait_for_batch(job, batch)
    bulk.close_job(job)

    if query:
        results = []
        for result in bulk.get_all_results_for_query_batch(batch):
            for row in json.load(IteratorBytesIO(result)):
                results.append({"Id": row["Id"]})
        return results

In [ ]:
def push_to_SF(df):
    query = f"""SELECT something
            FROM Table
            where StartTime__c > 
            {datetime.strftime(datetime.strptime(job['time'],'%d.%m.%Y'),"%Y-%m-%dT00:00:%S.000Z")}
            and StartTime__c < 
            {datetime.strftime(datetime.strptime(job['time'],'%d.%m.%Y'),"%Y-%m-%dT23:59:%S.000Z")}
            and isDeleted = False"""
    to_delete = sf_bulk_job(bulk, "Table", query=query)

    if len(to_delete) > 0:
        # delete old data from sf
        sf_bulk_job(bulk, "Table", delete=to_delete)

    if len(df) > 0:  # check that df isn't empty (eg Sundays)
        # create primary key from name and time
        df.loc[:, "Name"] = df["date"] + " " + df["time"] + " " + df["UserName__c"]

        # drop unused columns
        df.drop(["time", "date"], axis=1, inplace=True)

        # convert to dict for sf api
        inserts = df.to_dict(orient="records")

        # insert new data on sf
        sf_bulk_job(bulk, "Table", insert=inserts)

In [94]:
def gs_to_SF(date_range):
    for date in date_range:
        sheet_id = get_archived_shiftplan_sheet(date)
        
        #if the sheet is not in the archived then search in the current shiftplan sheet
        if sheet_id == 'sheet not found':
            sheet_id = 'gsheet_id'
            
        df = get_gs_to_df(sheet_id, date)
        df = df_to_pre_SF(df)
        df_list.append(df)
        push_to_SF(df)
        df.to_csv(f'df_{date.strftime("%d.%m.%Y")}.csv')

In [99]:
df_list = []
if __name__ == "__main__":
    gs_to_SF(pd.date_range('02-04-2019','02-05-2019'))
    print(df_list)